In [1]:
import math, random, torch, torch.nn as nn, torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm


def mobius_add(x, y, eps=1e-5):
    xy = (x * y).sum(dim=-1, keepdim=True)
    x2 = (x * x).sum(dim=-1, keepdim=True)
    y2 = (y * y).sum(dim=-1, keepdim=True)
    num = (1 + 2 * xy + y2) * x + (1 - x2) * y
    denom = 1 + 2 * xy + x2 * y2
    return num / (denom.clamp_min(eps))

def hyp_distance(x, y, eps=1e-5):
    x2 = (x * x).sum(dim=-1)
    y2 = (y * y).sum(dim=-1)
    diff2 = ((x - y) ** 2).sum(dim=-1)
    denom = (1 - x2) * (1 - y2)
    z = 1 + 2 * diff2 / denom.clamp_min(eps)
    return torch.acosh(z.clamp_min(1 + eps))

class MuRP(nn.Module):
    def __init__(self, num_ent, num_rel, dim):
        super().__init__()
        self.ent = nn.Embedding(num_ent, dim)
        self.rel = nn.Embedding(num_rel, dim)
        nn.init.uniform_(self.ent.weight, -0.001, 0.001)
        nn.init.uniform_(self.rel.weight, -0.001, 0.001)

    def forward(self, h, r, t):
        h_e = self.ent(h)
        r_e = self.rel(r).tanh()         
        t_e = self.ent(t)
        # translate: h ⊕ r
        h_r = mobius_add(h_e, r_e)
        dist = hyp_distance(h_r, t_e)
        return -dist                    

c:\Users\jerry\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_triples(path):
    triples = []
    with open(path) as f:
        for line in f:
            h,r,t = line.strip().split()[:3]
            triples.append((h,r,t))
    return triples

train_triples = load_triples("../graph_data/train.tsv")
dev_triples =  load_triples("../graph_data/dev.tsv")
test_triples =  load_triples("../graph_data/test.tsv")

entities = sorted({e for h,_,t in train_triples+dev_triples+test_triples for e in (h,t)})
relations = sorted({r for _,r,_ in train_triples+dev_triples+test_triples})
ent2id = {e:i for i,e in enumerate(entities)}
rel2id = {r:i for i,r in enumerate(relations)}


def encode(triples):
    return torch.tensor([[ent2id[h], rel2id[r], ent2id[t]] for h,r,t in triples], dtype=torch.long)

train_data = encode(train_triples)
dev_data   = encode(dev_triples)
test_data  = encode(test_triples)

In [3]:

batch_size = 512
train_loader = DataLoader(TensorDataset(train_data), batch_size=batch_size, shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MuRP(len(entities), len(relations), dim=100).to(device)
optimizer = AdamW(model.parameters(), lr=5e-4, weight_decay=1e-4)


def neg_sample(batch, n_ent):
    neg = batch.clone()
    mask = torch.rand(batch.size(0), device=batch.device) < 0.5
    neg[mask,0] = torch.randint(0,n_ent,(mask.sum().item(),),device=batch.device)
    neg[~mask,2] = torch.randint(0,n_ent,((~mask).sum().item(),),device=batch.device)
    return neg


In [4]:
def eval_rank(data_tensor, n_neg=100, ks=(1,3,10)):
    model.eval()
    ranks = []
    with torch.no_grad():
        for h,r,t in tqdm(data_tensor, desc='Eval', leave=False):
            h = torch.tensor([h], device=device)
            r = torch.tensor([r], device=device)
            t = torch.tensor([t], device=device)
            pos = model(h, r, t).item()
            scores = [pos]
            for _ in range(n_neg):
                if random.random()<0.5:
                    h2, t2 = random.randrange(len(entities)), t.item()
                else:
                    h2, t2 = h.item(), random.randrange(len(entities))
                scores.append(model(
                    torch.tensor([h2], device=device),
                    r,
                    torch.tensor([t2], device=device)
                ).item())
            rank = 1 + sum(s>pos for s in scores[1:])
            ranks.append(rank)
    ranks = torch.tensor(ranks, dtype=torch.float)
    mrr = (1.0/ranks).mean().item()
    hits = {f"Hits@{k}": (ranks<=k).float().mean().item() for k in ks}
    return mrr, hits

#

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MuRP(len(entities), len(relations), dim=100).to(device)
optimizer = AdamW(model.parameters(), lr=5e-4, weight_decay=1e-4)
best_dev_mrr = 0.0

for epoch in range(1, 201):
    model.train(); total_loss=0; total=0
    for batch in train_loader:
        pos = batch[0].to(device)
        neg = neg_sample(pos, len(entities))
        pos_s = model(pos[:,0], pos[:,1], pos[:,2])
        neg_s = model(neg[:,0], neg[:,1], neg[:,2])
        loss = - (F.logsigmoid(pos_s).mean() + F.logsigmoid(-neg_s).mean())
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        total_loss += loss.item()*pos.size(0); total+=pos.size(0)
    if epoch % 1 == 0:
        dev_mrr, _ = eval_rank(dev_data, n_neg=100)
        if dev_mrr > best_dev_mrr:
            best_dev_mrr = dev_mrr
            torch.save(model.state_dict(), 'best_murp.pt')
        print(f"Epoch {epoch:02d} | Train Loss: {total_loss/total:.4f} | Dev MRR: {dev_mrr:.4f}")


Epoch 01 | Train Loss: 1.3863 | Dev MRR: 0.0758


Epoch 02 | Train Loss: 1.3812 | Dev MRR: 0.2235


Epoch 03 | Train Loss: 1.3718 | Dev MRR: 0.4292


Epoch 04 | Train Loss: 1.3612 | Dev MRR: 0.5247


Epoch 05 | Train Loss: 1.3497 | Dev MRR: 0.5825


...

Epoch 196 | Train Loss: 0.7593 | Dev MRR: 0.6695


Epoch 197 | Train Loss: 0.7594 | Dev MRR: 0.6801


Epoch 198 | Train Loss: 0.7596 | Dev MRR: 0.6482


Epoch 199 | Train Loss: 0.7585 | Dev MRR: 0.6807


Epoch 200 | Train Loss: 0.7593 | Dev MRR: 0.6701


In [ ]:

model.load_state_dict(torch.load('best_murp.pt'))
test_mrr, test_hits = eval_rank(test_data.to(device), n_neg=100)
print(f"Test MRR: {test_mrr:.4f} | Hits@1: {test_hits['Hits@1']:.4f} | Hits@3: {test_hits['Hits@3']:.4f} | Hits@10: {test_hits['Hits@10']:.4f}")

C:\Users\jerry\AppData\Local\Temp\ipykernel_20684\314944752.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_murp.pt'))
           

Test MRR: 0.6024 | Hits@1: 0.5870 | Hits@3: 0.5870 | Hits@10: 0.6304
